In [0]:
import pandas as pd
from numpy import array
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

In [9]:
from google.colab import files
uploads = files.upload()

Saving Nairobi Office Price Ex.csv to Nairobi Office Price Ex.csv


In [29]:
data=pd.read_csv('Nairobi Office Price Ex.csv')
data.head()

,LOC,FUR,AMB,PROX_SCH,PROX_ROAD,PROX_MALL,WATER,HK_SER,SIZE,PRICE
0,karen,yes,serene,no,yes,yes,yes,yes,32.502345,31.707006
1,madaraka,yes,semi_serene,yes,yes,yes,no,no,53.426804,68.777596
2,karen,no,noisy,no,yes,yes,yes,yes,61.530358,62.562382
3,karen,yes,semi_serene,no,no,no,yes,yes,47.475640,71.546632
4,buruburu,no,semi_serene,no,yes,yes,yes,yes,59.813208,87.230925


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LOC        14 non-null     object 
 1   FUR        14 non-null     object 
 2   AMB        14 non-null     object 
 3   PROX_SCH   14 non-null     object 
 4   PROX_ROAD  14 non-null     object 
 5   PROX_MALL  14 non-null     object 
 6   WATER      14 non-null     object 
 7   HK_SER     14 non-null     object 
 8   SIZE       14 non-null     float64
 9   PRICE      14 non-null     float64
dtypes: float64(2), object(8)
memory usage: 1.2+ KB


In [31]:
nominal=['LOC','FUR','WATER','HK_SER']
ordinal=['AMB','PROX_SCH','PROX_ROAD','PROX_MALL']
numerical = ['SIZE']
target = ['PRICE']

# obtain categorical data
categorical_data = data.select_dtypes(include=['object']).copy()
categorical_cols = list(categorical_data.columns)

for col in categorical_cols:
  categorical_data[col] = categorical_data[col].astype('category')

# encode ordinal features using pandas
for feature in ordinal:
  categorical_data[feature] = categorical_data[feature].cat.codes

categorical_data.head()

,LOC,FUR,AMB,PROX_SCH,PROX_ROAD,PROX_MALL,WATER,HK_SER
0,karen,yes,2,0,1,1,yes,yes
1,madaraka,yes,1,1,1,1,no,no
2,karen,no,0,0,1,1,yes,yes
3,karen,yes,1,0,0,0,yes,yes
4,buruburu,no,1,0,1,1,yes,yes


In [32]:
# OH Encoding using pandas
data_copy = categorical_data.copy()
data_copy = pd.concat([pd.get_dummies(data_copy[nominal]),data_copy],axis=1) 
data_copy.drop(nominal,axis=1,inplace=True)
data_copy.head()

,LOC_buruburu,LOC_donholm,LOC_karen,LOC_langata,LOC_madaraka,FUR_no,FUR_yes,WATER_no,WATER_yes,HK_SER_no,HK_SER_yes,AMB,PROX_SCH,PROX_ROAD,PROX_MALL
0,0,0,1,0,0,0,1,0,1,0,1,2,0,1,1
1,0,0,0,0,1,0,1,1,0,1,0,1,1,1,1
2,0,0,1,0,0,1,0,0,1,0,1,0,0,1,1
3,0,0,1,0,0,0,1,0,1,0,1,1,0,0,0
4,1,0,0,0,0,1,0,0,1,0,1,1,0,1,1


In [33]:
# get dataset that includes target and numerical features and encoded categorical features
new_data = pd.concat([data_copy, data[numerical], data[target]], axis=1)
new_data.head()

,LOC_buruburu,LOC_donholm,LOC_karen,LOC_langata,LOC_madaraka,FUR_no,FUR_yes,WATER_no,WATER_yes,HK_SER_no,HK_SER_yes,AMB,PROX_SCH,PROX_ROAD,PROX_MALL,SIZE,PRICE
0,0,0,1,0,0,0,1,0,1,0,1,2,0,1,1,32.502345,31.707006
1,0,0,0,0,1,0,1,1,0,1,0,1,1,1,1,53.426804,68.777596
2,0,0,1,0,0,1,0,0,1,0,1,0,0,1,1,61.530358,62.562382
3,0,0,1,0,0,0,1,0,1,0,1,1,0,0,0,47.475640,71.546632
4,1,0,0,0,0,1,0,0,1,0,1,1,0,1,1,59.813208,87.230925


In [0]:
# # define dependent and independent variables
# all_features = list(new_data.columns)
# X = new_data.loc[:, all_features].values
# y = new_data.loc[:, ['Price']].values
# y

In [46]:
# define independent variables and conver to an array
X = new_data.drop("PRICE", 1)
X_array = X.to_numpy()
X_array

array([[ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         1.        ,  2.        ,  0.        ,  1.        ,  1.        ,
        32.50234527],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  1.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  1.        ,  1.        ,  1.        ,  1.        ,
        53.42680403],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  1.        ,
        61.53035803],
       [ 0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  1.        ,  0.        ,
         1.        ,  1.        ,  0.        ,  0.        ,  0.        ,
        47.47563963],
       [ 1.        ,  0.        ,  0

In [47]:
# standardize X using StandardScaler
X_standardized = StandardScaler().fit_transform(X_array)
X_standardized

array([[-0.40824829, -0.40824829,  1.34164079, -0.52223297, -0.40824829,
        -0.74535599,  0.74535599, -0.74535599,  0.74535599, -0.74535599,
         0.74535599,  0.89802651, -1.        ,  0.8660254 ,  0.8660254 ,
        -2.28520557],
       [-0.40824829, -0.40824829, -0.74535599, -0.52223297,  2.44948974,
        -0.74535599,  0.74535599,  1.34164079, -1.34164079,  1.34164079,
        -1.34164079, -0.3592106 ,  1.        ,  0.8660254 ,  0.8660254 ,
         0.40190153],
       [-0.40824829, -0.40824829,  1.34164079, -0.52223297, -0.40824829,
         1.34164079, -1.34164079, -0.74535599,  0.74535599, -0.74535599,
         0.74535599, -1.61644772, -1.        ,  0.8660254 ,  0.8660254 ,
         1.44255533],
       [-0.40824829, -0.40824829,  1.34164079, -0.52223297, -0.40824829,
        -0.74535599,  0.74535599, -0.74535599,  0.74535599, -0.74535599,
         0.74535599, -0.3592106 , -1.        , -1.15470054, -1.15470054,
        -0.36234362],
       [ 2.44948974, -0.40824829, -0

In [50]:
pca = PCA(n_components=2)
pc_s = pca.fit_transform(X_standardized)
#pca_data = pd.DataFrame(pc_s, columns=['PC1', 'PC2'])
final_data = pd.concat([pca_data, data[target]], axis=1)
final_data.head()

,PC1,PC2,PRICE
0,0.329734,-2.164692,31.707006
1,-2.569710,0.508011,68.777596
2,3.110888,-0.996581,62.562382
3,0.234592,-1.993858,71.546632
4,2.766518,0.325956,87.230925


In [51]:
pca.explained_variance_

array([4.21550601, 3.15426099])

In [53]:
pca.explained_variance_ratio_

array([0.2446499 , 0.18305979])

In [0]:
# final_data = 

In [0]:
# regressor = LassoCV()
# X = final_data.drop("PRICE",1)
# y = final_data["PRICE"]
# regressor = LassoCV()
# regressor.fit(X,y)